# **GaiaFHDs: Sternhaufen mit Gaia und Python**

## Willkommen!

In diesem Notebook werden wir Farben-Helligkeits-Diagramme (FHDs) von Sternhaufen erstellen und analysieren, um die Entfernung und das Alter dieser Sternhaufen abzuschätzen. Dafür nutzen wir Daten des Weltraumteleskops Gaia der ESA.

Die Nutzung des Notebooks werden wir besprechen. Es ist hier nicht jedes Detail schriftlich beschrieben. Melde Dich einfach, wenn etwas nicht klappt, und natürlich auch wenn du inhaltliche oder technische Fragen hast!

## Teil 1: Kleines Python-Tutorial

Damit du den Code in den nächsten Teilen besser nachvollziehen kannst, fangen wir mit einer kleinen Einleitung zu Python an.


Klicke in folgende "Zelle", und führe sie mit **control-enter** aus! Du kannst auch **shift-enter** nutzen, dann wird gleich die nächste Zelle ausgewählt.

In [ ]:
print("Hallo Welt!")

In [ ]:
# Alles hinter einem # ist ein Kommentar, der von Python ignoriert wird.

# Das hier ist ein "String", also eine Zeichenkette, die von Python als Text interpretiert wird:
t = "Hallo Welt!"

# In einem Notebook kannst du das print() oft weglassen.
# Eine Darstellung des letzten "Ausdrucks" einer Zelle wird automatisch ausgegeben, so wie hier:
t

In [ ]:
a = 6
b = 3
print("Kleine Rechnung:", (a + b - 2.2) / 2 )

# Wenn gewünscht kann man genau bestimmen, wie Zahlen formatiert werden sollen.
# Die modernste und eleganteste Lösung dafür in Python ist der sogenannte f-String:
print(f"Noch ne Rechnung: { a+b :.5f}") # Das .5f bedeutet, dass die Zahl mit 3 Nachkommastellen ausgegeben werden soll.

In [ ]:
# Variablen bleiben nach der Ausführung von Zellen erhalten!
# a und b "gibt" es also weiterhin:

a / b

Für praktisch all Aufgaben gibt es in Python "Module", das sind Programmierbibliotheken. Um sie zu nutzen, muss man sie vorher einmal *importieren*.

Erstes Beispiel: für numerische Rechnungen nutzt man **numpy**, und importiert es als np (... ist kürzer zu schreiben), so: 

In [ ]:
import numpy as np

# Alles, was zu numpy gehört, fängt nun mit "np." an:
x = 2.0
y = np.sqrt(x) # sqrt steht für "square root"

print("Die Quadratwurzel von", x, "ist ungefähr", y)

print("Pi ist ungefähr", np.pi)


Die wichtigste Eigenschaft von numpy ist das rechnen mit sogenannten "Arrays". Ein Array ist eine Art Liste (oder Tabelle) von Zahlen.

So können wir ein Array selber bauen:

In [ ]:
a = np.array([1.5, 4.2, 2.0, 1.0])

# Und so kommen wir an einzelne Elemente des Arrays ran:
print("Das erste Element von a ist", a[0])
print("Das zweite Element von a ist", a[1])
print("Das letzte Element von a ist", a[-1])

Und jetzt kommt der Clou: wenn man mit einem Array rechnet, werden viele Operationen automatisch auf *alle* Zahlen dieses Arrays angewandt. Das Ergebnis so einer Rechnung ist oft selber ein Array. Probiere folgende Beispiele aus:

In [ ]:
b = a + 2.0  # b ist nun auch ein Array mit 4 Zahlen...
print(b)

b = np.sqrt(a)  # ... und hier ist b die Quadratwurzel von a
print(b)

Andere Funktionen von numpy können ein Array "zusammenfassen", und nur eine Zahl ausgeben. Zum Beispiel:

In [ ]:
print(np.mean(a)) # Mittelwert von a

c = np.max(b) # Maximalwert von b
print(c)

Als nächstes Beispiel schauen wir uns kurz das Modul **matplotlib** an, mit dem man Diagramme erstellen kann. Mit dieser Bibliothek können alle Aspekte eines Diagramms bis ins letzte Detail kontrolliert werden, und trotzdem bleibt das erstellen von einfachen Diagrammen so unkompliziert wie möglich.



In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt # Wir werden matplotlib immer so importieren

# Matplotlib funktioniert mit numpy Arrays, wir bauen uns also ein Beispiel:
x = np.linspace(0, 5, 50) # so bekommt man ein Array mit 50 regelmässig verteilte Zahlen zwischen 0 und 5
y = np.sqrt(x) # Das Array y enthält also die Quadratwurzeln der Werte von x

# Und jetzt matplotlib:
plt.figure() # Lege eine neues Diagramm an
plt.plot(x, y, "b.") # Zeichne (x, y) als blaue Punkte ein ("b." steht für "blue dot")
plt.show() # Zeige das Diagramm an

Matplotlib-Diagramme sind interaktiv. Bewege die Maus über das Diagramm, und Klicke auf das Kreuz-Symbol oben links. Jetzt kannst du mit der linken Maustaste die Achsen verschieben, und mit der rechten Maustaste die Skala der Achsen ändern.

Spiele ein wenig damit herum:

 - Ersetze mal `sqrt` durch `sin`.
 - Ersetzte `"b."` durch `"r-"`, oder lasse das ganz weg.
 - Du kannst die Funktion `plt.plot` auch mehrmals aufrufen (natürlich am besten mit unterschiedlichem Inhalt).
 - Füge die Zeile `plt.xlabel("Das ist die x-Achse")` ein, vor dem `plt.show()`.

## Teil 2: Laden der Daten

In diesem Teil laden wir die Gaia-Daten eines Sternhaufens aus einer Datenbank. 

Als erstes importieren wir jetzt alle Module, die wir heute benötigen werden. Führe die folgende Zelle einfach einmal aus.

In [ ]:
import warnings

import urllib.request
import pathlib
import pickle

import numpy as np
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

import pyvo

import astropy
import astropy.table
from astropy import units as u

from ipyaladin import Aladin, Marker

Jetzt wählen wir einen Sternhaufen aus, hier sind einige Vorschläge.

- Kugelsternhaufen: M2, M3, M4, M5, M10, M12, M13, M15, M22, M28, M53, M92, NGC104, NGC5139
- Offene Sternhaufen: M37, M38, M46, M67, NGC3293, NGC3532, NGC3766 


In [ ]:
sternhaufen_name = "NGC3766"
gaia_radius_grad = 0.5 # Radius in Grad, in dem wir den Katalog laden werden. Lass den Wert zunächst unverändert.

Spezialisierte Astronomie-Module liefern uns Himmelskoordinaten:

In [ ]:
sternhaufen_position = astropy.coordinates.SkyCoord.from_name(sternhaufen_name)
dec_grad = sternhaufen_position.dec.to_value(u.deg)
ra_grad = sternhaufen_position.ra.to_value(u.deg)

galaktische_breite = sternhaufen_position.galactic.b.to_value(u.deg)

print(f"Koordinaten (in Grad) von {sternhaufen_name}: DEC = {dec_grad:.4f}, RA = {ra_grad:.4f}")
print(f"Und seine galaktische Breite (in Grad): {galaktische_breite:.2f}")

Zunächst laden wir ein Bild des sternhaufens, aus einer Durchmusterung. Wir nutzen dazu eine Werkzeug namens "Aladin", aus einem Python-Modul das wir bereits importiert haben.
Das Angezeigte Bild ist interaktiv, du kannst z.B. raus- und rein-zoomen!

In [ ]:
aladin = Aladin(target=sternhaufen_name, fov=3.0, survey="P/DSS2/color", height=600)
aladin

In [ ]:
# Wir können auch den Radius darstellen, in dem wir den Katalog laden wollen:
aladin.add_graphic_overlay_from_stcs(f"Circle ICRS {ra_grad} {dec_grad} {gaia_radius_grad}", color="yellow")

# Nachdem du diese Zelle ausgeführt hast, bewege die Maus nochmal über das Aladin-Fenster (direkt hier drüber).
# Der Radius sollte dann erscheinen.

In der folgenden Zelle werden wir nun einen Katalog der Sterne um die Position des Sternhaufens laden, aus einer Gaia-Datenbank. Wir brauchen hier nur Sterne bis zur 17. Magnitude.

Diesen Katalog schreiben wir in eine Text-Datei, um alles so einfach und flexibel wie möglich zu halten.

In [ ]:
# Dateinamen, unter dem wir den Katalog speichern wollen:
dateiname_katalog = f"katalog_um_{sternhaufen_name}.tsv"

# Wir wollen den Katalog nur laden, wenn das bisher noch nicht gemacht wurde.
# Daher prüfen wir zuerst, ob die Datei schon existiert.
if pathlib.Path(dateiname_katalog).exists():
    raise FileExistsError(
        f"Den Katalog {dateiname_katalog} gibt es schon, bitte lösche ihn wenn du ihn wirklich neu laden willst."
    )

# Woher wir die Daten laden wollen:
tap_service = pyvo.dal.TAPService("https://gaia.ari.uni-heidelberg.de/tap")

# Und wir beschreiben, in einer speziellen Sprache, welche Daten wir wollen.
# In den ersten Zeilen, nach SELECT, geben wir die namen der gewünschten "Spalten" (also Messungen) an.
# Dann folgt die Angabe der Himmelsregion um den Sternhaufen.
tap_anfrage = f"""SELECT ra, dec, pmra, pmdec,
phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, parallax, parallax_error
FROM gaiadr3.gaia_source
WHERE 1 = CONTAINS(POINT('ICRS', ra, dec),
                   CIRCLE('ICRS', {ra_grad}, {dec_grad}, {gaia_radius_grad}))
AND phot_g_mean_mag < 19.0
"""
print("Die Anfrage an der Server lautet:")
print(tap_anfrage)

# Nun stellen wir diese Anfrage, und bekommen (wenn alles klappt) die Daten.
print("Die Anfrage wird nun gestellt.")
print("Es kann einen Moment dauern, bis die Daten angekommen sind.")
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", module='astropy.*')
    tap_daten = tap_service.run_sync(tap_anfrage)

gaia_daten_tabelle = tap_daten.to_table()

print("Wir haben den Katalog bekommen. Übersicht der Spalten:")
print(gaia_daten_tabelle.info)
# Diese Übersicht ist nützlich, daher schreiben wir sie in eine Datei:
with open(f"info_katalog_um_{sternhaufen_name}.txt", "w") as datei:
    datei.write(str(gaia_daten_tabelle.info))

# Jetzt schreiben wir den Katalog in eine einfache Text-Datei,
# als eine große Tabelle, im Format "tab-separated values", oft tsv genannt.
gaia_daten_tabelle.write(dateiname_katalog, format="ascii.tab", overwrite=True)
print("Fertig!")

In [ ]:
# Nur zur Info, so könntest du die soeben geschriebenen Dateien aus Python heraus löschen:
#pathlib.Path.unlink(f"katalog_um_{sternhaufen_name}.tsv")
#pathlib.Path.unlink(f"info_katalog_um_{sternhaufen_name}.txt")

Jetzt lesen wir diesen Katalog aus der Datei ein (wie du siehst geht das z.B. mit einer Funktion von Numpy):

In [ ]:
katalog = np.genfromtxt(f"katalog_um_{sternhaufen_name}.tsv", delimiter="\t", names=True)

Und visualisieren ihn in Aladin! Führe die folgende Zelle aus, und sieh dir das Ergebnis in dem Aladin-Fenster weiter oben an!

In [ ]:
aladin.add_table( # Wir fügen eine Darstellung des Katalogs in Aladin hinzu.
    astropy.table.Table(katalog),
    shape="circle", color="red", ra_field="ra", dec_field="dec"
)

Übrigens ist `katalog` ein numpy Array, in diesem Fall mit 2 Dimensionen: jede "Zeile" steht für einen Stern, jede "Spalte" für einen gemessenen Parameter. Du kannst dir dieses Array natürlich hier im Notebook anzeigen lassen, und auch damit rechnen. Mithilfe der Spalten-Namen kommst du leicht an den Inhalt einzelner Spalten, wie in folgendem Beispiel.

In [ ]:
print("Namen der Spalten: ", katalog.dtype.names)  # Spaltennamen des Katalogs

print("Mittelwert der Werte der Spalte 'ra': ", np.mean(katalog['ra']) )

## Teil 3: Auswahl der Sterne des Sternhaufens

Anhand der von Gaia gemessenen Eigenbewegungen können wir die Sterne identifizieren, die *tatsächlich* zu dem Haufen gehören.

Dazu erstellen wir ein Diagramm der von Gaia gemessenen Eigenbewegungen (Englisch: "proper motion", abgekürzt "pm") der Sterne, mit Matplotlib.
Dafür verwenden wir die Spalten `pmra` und `pmdec` als x- und y-Achse, die die Eigenbewegungen der Sterne entlang der Himmelskoordinaten enthalten.

In [ ]:
plt.figure()
plt.plot(katalog["pmra"], katalog["pmdec"],
    marker=".", markersize=1, linestyle="None")
plt.axis("equal")  # Damit die Achsen gleich skaliert werden

plt.title(f"Gaia-Eigenbewegungen im Himmelsauschnitt um {sternhaufen_name}")
plt.xlabel("Eigenbewegung entlang RA [mas/yr]")  # mas/yr = Milliarcsekunden pro Jahr
plt.ylabel("Eigenbewegung entlang Dec [mas/yr]")

plt.tight_layout()
plt.show()

Kannst du in dem Diagramm einen "Haufen" finden? Wenn ja, was könnte das sein?
Wenn nicht sind in deinem Katalog vielleicht einfach zu wenige Feldsterne, das ist aber auch kein Problem.

Anhand ihrer Eigenbewegung wollen wir nun aus dem Katalog die Sterne auswählen, die tatsächlich zu dem Sternhaufen gehören.
Das können wir mit folgender Zelle machen, **in der du die Werte `pmra_min`, `pmra_max`, `pmdec_min`, `pmdec_max` anpassen musst!**

In [ ]:
# Grenze einen Bereich der Eigenbewegungen ein (in mas/yr), der dem Sternhaufen entsprechen könnte.
# Ersetze dafür die np.inf durch Werte, die du im obigen Diagramm abliest.
(pmra_min, pmra_max) = (-np.inf, np.inf)
(pmdec_min, pmdec_max) = (-np.inf, np.inf)

# Wir machen einen neuen Katalog, der nur die Sterne enthält, die in dem eingegrenzten Bereich
# der Eigenbewegungen liegen.
katalog_auswahl = np.extract((katalog["pmra"] >= pmra_min) & (katalog["pmra"] <= pmra_max) &
                     (katalog["pmdec"] >= pmdec_min) & (katalog["pmdec"] <= pmdec_max), katalog)

print(f"katalog_auswahl enthält {len(katalog_auswahl)} von {len(katalog)} Sternen.")

Wir können die Auswahl überprüfen, indem wir uns die Verteilung der ausgewählten Sterne am Himmel ansehen.
Führe dazu folgende Zelle aus, und scrolle dann wieder hoch zum Aladin-Fenster. Die ausgewählten Sterne sind dort nun in *grün* markiert.
Hat die Auswahl geklappt?

In [ ]:
aladin.add_table( # Wir fügen eine Darstellung des Katalogs in Aladin hinzu.
    astropy.table.Table(katalog_auswahl),
    shape="circle", color="green", ra_field="ra", dec_field="dec"
)

## Teil 4: Das Farben-Helligkeits-Diagramm (FHD)

Wir erstellen das FHD des Sternhaufens, und vergleichen es mit simulierten Sternen, um Entfernung und Alter des Sternhaufens abschätzen zu können.

Alles, was wir für das FHD benötigen, befindet sich in `katalog_auswahl`. Folgende Spalten werden wir nun nutzen:

- "phot_bp_mean_mag": entspricht der scheinbaren Magnitude in einem **blauen** Filter
- "phot_rp_mean_mag": entspricht der scheinbaren Magnitude in einem **roten** Filter

Die *Farbe* können wir berechen, indem wir die Differenz zwischen diesen Spalten bilden.
Wir rufen also `plt.plot()` auf, und geben als erstes die Farbe (x-Achse), und dann die scheinbare Helligkeit im roten Filter (y-Achse) an:

In [ ]:
plt.figure()
plt.plot(
    katalog_auswahl["phot_bp_mean_mag"] - katalog_auswahl["phot_rp_mean_mag"], # Farbe: B - R
    katalog_auswahl["phot_rp_mean_mag"], # Scheinbare Helligkeit: R
    marker=".", markersize=2, linestyle="None", alpha=0.3, rasterized=True
    )

# Wir ändern wir die Richtung der y-Achse, damit helle Sterne (also kleine Magnituden) oben sind.
plt.gca().invert_yaxis()

# Und Beschriften das Diagram:
plt.title(f"Gaia-FHD von {sternhaufen_name}")
plt.xlabel("Farbe B - R [mag]")
plt.ylabel("Scheinbare Helligkeit R [mag]")

plt.tight_layout()
plt.show()

Dieses Diagramm wollen wir nun mit "simulierten" Sternen vergleichen.
Führe dazu die folgenden 3 Zellen einfach aus. Den Code musst du zunächst nicht lesen, du kannst später darauf zurück kommen. 

In [ ]:
# Wir laden eine Datei mit Ergenissen der Simulationen (sogenannte Isochronen).

local_isochrones_filepath = pathlib.Path("MIST_Gaia_isochrones.pickle")

isochrones_url = "https://uni-bonn.sciebo.de/s/twnf8ZiN66RySCT/download/MIST_Gaia_isochrones.pickle"
if not local_isochrones_filepath.exists():
    urllib.request.urlretrieve(isochrones_url, local_isochrones_filepath)

with open(local_isochrones_filepath, 'rb') as f:
    (isochrones, vrots, metallicities, logages) = pickle.load(f)

In [ ]:
# Wir definieren Funktionen die uns ermöglichen, die "simulierten Sterne" mit den echten Beobachtungen zu vergleichen.

def extinction_law(A_V):
    """
    Returns extinction values for the different Gaia filters, based on some reference total extinction A_V

    The particular values we use here are from Wang and Chen (2019) ApJ 877 116:
    https://iopscience.iop.org/article/10.3847/1538-4357/ab1c61
    """
    return {"G": A_V*0.8, "RP": A_V*0.589, "BP":A_V*1.002 } # Table 3 of Wang and Chen (2019)



def mag_absolute_to_apparent(m, distance, extinction):
    """
    Function that corrects absolute magnitudes by distance modulus and extinction, to get apparent magnitudes
    
    m: absolute magnitude(s)
    distance: in pc
    extinction: in mag
    """
    return m + 5.0*np.log10(distance) - 5 + extinction 


def get_apparent_isochrone(vrot_index, metallicity_index, age_index, distance=10, A_V=0):
    """
    Helper function to get a specific isochrone at a specific distance and total extinction
    """
    
    # Computing the extinctions:
    extinctions = extinction_law(A_V)
    
    # Obtaining the corrected magnitudes:
    iso_app_g = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["G"], distance, extinctions["G"])
    iso_app_r = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["BP"], distance, extinctions["BP"])
    iso_app_b = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["RP"], distance, extinctions["RP"])
    
    # A fancy string describing the isochrone, with some LaTeX formatting:
    label_string = f"Alter {(10.0**logages[age_index])/(1e9):.2f} Gyr, " \
        + f"[Fe/H] = {metallicities[metallicity_index]:.2f}, " \
        + r"$v/v_{\mathrm{crit}}$ = " + f"{vrots[vrot_index]}, " \
        + f"$d$ = {distance:.0f} pc, " \
        + r"$A_{\mathrm{V}}$ = " + f"{A_V:.2f}"

    return (iso_app_g, iso_app_r, iso_app_b, label_string)


In [ ]:
# Diese Zelle baut eine interaktive Matplotlib-Figur. Führe sie einfach aus.

# Initial position of sliders:
init_distance = 10.0
init_age_index = 50
init_metallicity_index = 12
init_A_V = 0.0
init_vrot_index = 0

fig, ax = plt.subplots(figsize=(10, 6))
fig.subplots_adjust(left=0.30, bottom=0.1) # making room for the sliders

# Plotting the observed data
cbar = ax.plot(
    katalog_auswahl["phot_bp_mean_mag"] - katalog_auswahl["phot_rp_mean_mag"], # Farbe: B - R
    katalog_auswahl["phot_rp_mean_mag"], # Scheinbare Helligkeit: R
    marker=".", markersize=2, linestyle="None", alpha=0.3, rasterized=True
)

# Plotting the isochrone
(g, bp, rp, label_string) = get_apparent_isochrone(init_vrot_index, init_metallicity_index, init_age_index, init_distance, init_A_V)
line, = ax.plot(bp-rp, rp, color="red", linewidth=1.0) 
text = ax.text(0.5, 0.95, label_string, color="red", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

# The function to be called anytime a slider or button is changed
def update(val):
    vrot_index = vrot_button.index_selected
    metallicity_index = metallicity_slider.val
    age_index = age_slider.val
    distance = dist_slider.val
    A_V = ext_slider.val
    (g, bp, rp, label_string) = get_apparent_isochrone(vrot_index, metallicity_index, age_index, distance, A_V)
    line.set_xdata(bp-rp)
    line.set_ydata(rp)
    text.set_text(label_string)
    fig.canvas.draw_idle()

# Slider to control the distance
axdist = fig.add_axes([0.05, 0.25, 0.0225, 0.6])
dist_slider = matplotlib.widgets.Slider(
    ax=axdist,
    label=r"$d$ [pc]",
    valmin=10,
    valmax=12000,
    valinit=init_distance,
    orientation="vertical"
)

# Slider to control the age
axage = fig.add_axes([0.09, 0.25, 0.0225, 0.6])
age_slider = matplotlib.widgets.Slider(
    ax=axage,
    label=r"$i_{\mathrm{Alter}}$",
    valmin=0,
    valmax=len(logages)-1,
    valstep=1,
    valinit=init_age_index,
    orientation="vertical"
)

# Slider to control the metallicity
axmetal = fig.add_axes([0.13, 0.25, 0.0225, 0.6])
metallicity_slider = matplotlib.widgets.Slider(
    ax=axmetal,
    label=r"$i_{\mathrm{[Fe/H]}}$",
    valmin=0,
    valmax=len(metallicities)-1,
    valstep=1,
    valinit=init_metallicity_index,
    orientation="vertical"
)

# Slider to control the extinction
axext = fig.add_axes([0.17, 0.25, 0.0225, 0.6])
ext_slider = matplotlib.widgets.Slider(
    ax=axext,
    label=r"$A_{\mathrm{V}}$",
    valmin=0.0,
    valmax=3.0,
    valinit=init_A_V,
    orientation="vertical"
)

# Radio buttons to control the rotation
axvrot = fig.add_axes([0.05, 0.1, 0.14, 0.08])
vrot_button = matplotlib.widgets.RadioButtons(
    ax=axvrot, 
    labels=("No rotation", r"$v/v_{\mathrm{crit}} = 0.4$"),
    active=init_vrot_index
)

# Register the update function with each slider and button:
age_slider.on_changed(update)
dist_slider.on_changed(update)
metallicity_slider.on_changed(update)
ext_slider.on_changed(update)
vrot_button.on_clicked(update)

# Finalize plot
ax.invert_yaxis()
ax.set_xlabel("Farbe B - R [mag]")
ax.set_ylabel("Scheinbare Helligkeit R [mag]")
ax.set_title(f"Gaia-FHD von {sternhaufen_name}")
plt.show()

Anhand der 4 "Schieberegler" kannst du nun die simulierten Sterne (dargestellt durch die rote linie) beeinflussen, um sie mit den Beobachtungen vergleichen zu können. Die Schieberegler haben folgende Bedeutung:

- Der erste parameter *d* ist die **Entfernung** der simulierten Sterne, in parsec (abgekürzt pc). Erhöhe mal die Entfernung! Wie du siehst, macht das die Sterne dunkler, ohne ihre Farbe zu verändern.
- Der zweite Regler beeinflusst das **Alter** der simulierten Sterne. Dieses wird dir in der roten Zeile angezeigt, in Milliarden Jahren (Gigayear, abgekürzt Gyr).
- Der dritte Regler beeinflusst die **Metallizität** der simulierten Sterne, also welchen Anteil die schwerere Elemente (alles außer Wasserstoff und Helium) an ihrer Masse haben. Für einen offenen Sternhaufen fangt am besten mit der Standart-Stellung an, das entspricht der Metallizität der Sonne. Für einen Kugelsternhaufen stellt ihn am Anfang auf [Fe/H] = -2.0, das entspricht einem Hunderstel der Metallizität der Sonne.
- Der vierte Regler beeinflusst die sogenannte **Extinktion**, das heisst wie viel Staub zwischen uns und den Sternen liegt. Der Wert wird durch den Parameter *A_V* (in der roten Zeile) angegeben, in Magnituden. Staub verdunkelt die Sterne, und verändert auch ihre Farbe: sie werden röter. 

Kannst du insbesondere die Entfernung, das Alter, und eventuell auch Staub und Metallizität so einstellen, dass die simulierten Sterne über den Beobachtungen liegen?

## Teil 5: Zusammenführen der Ergebnisse

Wir sammeln die Ergebnisse verschiedener Sternhaufen in einer Tabelle, um daraus z.B. etwas über unsere Milchstraße zu lernen.


Wenn du auf folgenden Link klickst, öffnet er sich in einem neuen Tab des Browsers.

**Du kannst dann die Tastenkombi control-tab nutzen, um zwischen den zwei Tabs des Browsers hin und her zu wechseln.**

https://docs.google.com/spreadsheets/d/1W0GKbjyw9aVHF4K5B1VOvJqouxk3H_Mv9EgN-Bxl35Y/

Tragt eure Werte für Alter, Entfernung, Metallizität und Extinktion in die Tabelle ein, und auch die galaktische Breite (wurde in der dritten Zelle von Teil 2 ausgegeben).

## Weiteres



Eine etwas systematischere Einleitung zu Gaia, FHDs, Sternentwicklung, und auch zu Python findest du im "Project Andromeda": https://astro.uni-bonn.de/andromeda/ 

## Notlösungen, falls etwas nicht funktioniert

In [ ]:
# Falls die Gaia-Datenbank nicht erreichbar ist, kannst du mit dieser Zelle die Daten
# für einige Sternhaufen aus einer anderen Quelle laden.
# Entferne dafür die Kommentarzeichen (#) vor den Zeilen.


#local_archive_path = pathlib.Path("GaiaFHDs_backup_Kataloge.tgz")

#archive_url = "https://uni-bonn.sciebo.de/s/mD6NKwcHbKEdBsE/download/GaiaFHDs_backup_Kataloge.tgz"
#if not local_archive_path.exists():
#    urllib.request.urlretrieve(archive_url, local_archive_path)

#import os
#os.system(f"tar xvf {local_archive_path}")

In [ ]:
# Tests

from ipyaladin import __version__, __aladin_lite_version__
print("version:", __version__, "running Aladin Lite:", __aladin_lite_version__)